In [ ]:
!pip install datasets
!pip install Transformers
!pip install sentencepiece
!pip install adapter-Transformers
!pip install rouge
!pip install rouge_score

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, AutoTokenizer,AutoModel, T5Config, AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset, list_datasets,load_metric

In [ ]:
# list_datasets()

In [ ]:
ds_test = load_dataset('ccdv/cnn_dailymail','3.0.0' ,split='test[:10]')

def add_prefix(example):
  example['article'] = 'summarize: ' + example['article']
  return example

ds_test = ds_test.map(add_prefix)  

Downloading:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


0ex [00:00, ?ex/s]

In [ ]:
# Failed as it's not a pd it is Dataset object so 
# ds_test['article'] = 'summarize: ' + ds_test['article']
# ds_test['highlights'] = '<pad>' + ds_test['highlights']

In [ ]:
print("Size of validation :",len(ds_test['article']))

Size of validation : 10


In [ ]:
test_texts, test_labels = ds_test['article'], ds_test['highlights']

In [ ]:
# Creating model with adapter 
from transformers import T5Config, T5Model, AdapterConfig, AutoModelWithHeads, get_adapter_info, AutoModelForSeq2SeqLM

model = AutoModel.from_pretrained('t5-small')

# adapter

adapter_name = model.load_adapter('/content/drive/MyDrive/t5-small-train-adapter-cnn/cnn_dailymail')
model.save_pretrained('my-T5-small')

# print(model.config)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
print(model.config)

T5Config {
  "_name_or_path": "t5-small",
  "adapters": {
    "adapters": {
      "cnn_dailymail": "9076f36a74755ac4"
    },
    "config_map": {
      "9076f36a74755ac4": {
        "adapter_residual_before_ln": false,
        "cross_adapter": false,
        "inv_adapter": null,
        "inv_adapter_reduction_factor": null,
        "leave_out": [],
        "ln_after": false,
        "ln_before": false,
        "mh_adapter": false,
        "non_linearity": "relu",
        "original_ln_after": true,
        "original_ln_before": true,
        "output_adapter": true,
        "reduction_factor": 16,
        "residual_before_ln": true
      }
    },
    "fusion_config_map": {},
    "fusions": {}
  },
  "architectures": [
    "T5Model"
  ],
  "attention_probs_dropout_prob": 0.1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,

In [ ]:
print(AdapterConfig)

<class 'transformers.adapters.configuration.AdapterConfig'>


In [ ]:
%%time
# with adapter

model = AutoModelForSeq2SeqLM.from_pretrained('/content/my-T5-small')
model.set_active_adapters(adapter_name)

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

CPU times: user 1.27 s, sys: 315 ms, total: 1.58 s
Wall time: 5.21 s


In [ ]:
#Source : https://huggingface.co/docs/transformers/model_doc/t5
inputs = tokenizer([sentence for sentence in test_texts], return_tensors="pt",truncation=True, padding=True)

In [ ]:
# labels = tokenizer([sentence for sentence in test_labels], return_tensors="pt",truncation=True, padding=True).input_ids

In [ ]:
# test_modeling_t5

In [ ]:
# loss = model(inputs,labels= labels).loss

In [ ]:
# mtf_score = -(labels.shape[-1] * loss.item())
# print(mtf_score)

In [ ]:
# model.config

In [ ]:
# inputs

In [ ]:
%%time
output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    num_beams=1,
)

CPU times: user 6.92 s, sys: 314 ms, total: 7.23 s
Wall time: 7.57 s


In [ ]:
final_output = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [ ]:
# final_output

In [ ]:
rouge = load_metric('rouge')
result = rouge.compute(predictions=final_output, references=test_labels)
# print(result['rouge1'].mid.fmeasure)

result1 = {key: value.low.fmeasure * 100 for key, value in result.items()}
print(f'Low : {result1}')

result2 = {key: value.mid.fmeasure * 100 for key, value in result.items()}
print(f'Mid : {result2}')

result3 = {key: value.high.fmeasure * 100 for key, value in result.items()}
print(f'High : {result3}')
# .........................................................................
# Individual score :

# from rouge import Rouge

# rouge = Rouge()
# rouge.get_scores(final_output, test_labels, avg=True)

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

Low : {'rouge1': 24.05341810792376, 'rouge2': 6.431451210826214, 'rougeL': 18.59177725663147, 'rougeLsum': 21.827739948026295}
Mid : {'rouge1': 30.55740839230993, 'rouge2': 13.413401739872329, 'rougeL': 25.72941215645671, 'rougeLsum': 27.89156417728317}
High : {'rouge1': 39.12707626910413, 'rouge2': 24.194693656017183, 'rougeL': 35.38447822170327, 'rougeLsum': 36.82596287224365}


In [ ]:
name = 'Tank'

In [ ]:
result2

In [ ]:
# update dictionary
result2.update( [('Austria', 43),('Name',name)] )

In [ ]:
result2

In [ ]:
import csv

field_name = ['rouge1','rouge2','rougeL','rougeLsum']

print(type(field_name))
with open('./result.csv','a') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = field_name)  
  writer.writeheader()
  writer.writerows([result2])
  csvfile.close()

In [ ]:
import csv

field_name = ['r1','r2','r3']
dictionaryobj= [{'r1':'1','r3':'10','r2':'5'}]
print(type(dictionaryobj))
with open('./output.csv','a') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = field_name)  
  # writer.writeheader()
  writer.writerows(dictionaryobj)
  csvfile.close()

In [ ]:
import csv

field_name = ['r1','r2','r3']
dictionaryobj= [{'r1':'11','r2':'15','r3':'11'}]
print(type(dictionaryobj))
with open('./output.csv','a') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = field_name)  
  # writer.writeheader()
  writer.writerows(dictionaryobj)
  csvfile.close()